In [ ]:
# mount google colab

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages

!pip install transformers datasets evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

right_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_right_fulltext.csv')
right_dataset = right_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
right_dataset = right_dataset.train_test_split(test_size=0.2)

# Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["full_text_cleaned"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_right_dataset = right_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=right_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/5489 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1702 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1482 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1367 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (593 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1373 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (733 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1085 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (7358 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (757 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# chunk size for concatenation
block_size = 128

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset
lm_dataset = tokenized_right_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

Map (num_proc=4):   0%|          | 0/5489 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1373 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# add pad token to tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

# Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("bert-base-uncased", is_decoder = True)

In [ ]:
#finetune the pretrained model on the dataset
training_args = TrainingArguments(
    output_dir="bert_right_fulltext_cleaned_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=6
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 40.58


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file runs/Jul26_11-20-05_92176356e603/events.out.tfevents.1690370411.92176356e603.6647.1:   0%|        …

Upload file runs/Jul26_11-20-05_92176356e603/events.out.tfevents.1690375212.92176356e603.6647.2:   0%|        …

To https://huggingface.co/tobijen/bert_right_fulltext_cleaned_torch
   235d449..74b228a  main -> main

   235d449..74b228a  main -> main

To https://huggingface.co/tobijen/bert_right_fulltext_cleaned_torch
   74b228a..d355a0d  main -> main

   74b228a..d355a0d  main -> main



'https://huggingface.co/tobijen/bert_right_fulltext_cleaned_torch/commit/74b228a5139eccbc90ede461d1f3eac0205b7685'

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/bert_right_fulltext_cleaned_torch")

# Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
from transformers import pipeline
# generate text with the pipeline module from the transformers library
generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/bert_right_fulltext_cleaned_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=100)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is
[{'generated_text': 'Gay marriage is not a right. ” the president said the issue is “ not a matter of religion. ” “ i think it ’ s a matter of religion, ” he said. “ i think it ’ s a matter of religion. ” the president said he would not support the ban, but he would not support it. “ i don ’ t think it ’ s a matter of religion, ” he said. “ i think it ’ s a matter of religion. i think it ’ s a matter of'}]
social_2  =>  Abortion is
[{'generated_text': 'Abortion is a big deal, ” said sen. john cornyn, texas republican. “ it ’ s a big deal. ” the senate is expected to vote on the bill on tuesday. the senate is expected to vote on the bill on monday. the senate is expected to vote on the bill on monday. the u. s. economy added a staggering 6. 5 percent annual growth in the second quarter, the largest jump since the great recession in 2008. the unemployment rate ticked up to 3.'}]
education_1  =>  Public education is
[{'generated_text': 'Public education is a 

In [ ]:
generated_text_dict

{'social_1': 'Gay marriage is not a right. ” the president said the issue is “ not a matter of religion. ” “ i think it ’ s a matter of religion, ” he said. “ i think it ’ s a matter of religion. ” the president said he would not support the ban, but he would not support it. “ i don ’ t think it ’ s a matter of religion, ” he said. “ i think it ’ s a matter of religion. i think it ’ s a matter of',
 'social_2': 'Abortion is a big deal, ” said sen. john cornyn, texas republican. “ it ’ s a big deal. ” the senate is expected to vote on the bill on tuesday. the senate is expected to vote on the bill on monday. the senate is expected to vote on the bill on monday. the u. s. economy added a staggering 6. 5 percent annual growth in the second quarter, the largest jump since the great recession in 2008. the unemployment rate ticked up to 3.',
 'education_1': 'Public education is a good thing for the american people. ” the president said the administration is “ deeply concerned ” with the issu

In [ ]:
# add the generated text to a json file, wich is used for the evaluation


import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "bert_right_fulltext_cleaned": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)